In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb 
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('../data/EV_data.csv')
df.head()

,read_date,interval_1,interval_2,interval_3,interval_4,interval_5,interval_6,interval_7,interval_8,interval_9,...,interval_41,interval_42,interval_43,interval_44,interval_45,interval_46,interval_47,interval_48,id,label
0,3/1/2021,0.0625,0.0500,0.0687,0.0750,0.0687,0.0500,0.0625,0.0687,0.0687,...,0.0812,0.0687,0.0687,0.0562,0.0562,0.0687,0.0687,0.0625,1,1
1,3/2/2021,0.0625,0.0500,0.0687,0.0625,0.0625,0.0562,0.0562,0.0625,0.0687,...,0.1375,0.0750,0.0687,0.0625,0.0625,0.0562,0.0625,0.0625,1,1
2,3/3/2021,0.0625,0.0687,0.0500,0.0562,0.0687,0.0625,0.0625,0.0625,0.0500,...,0.1875,0.1062,0.0750,0.0687,0.0687,0.0625,0.0562,0.0562,1,1
3,3/4/2021,0.0625,0.0625,0.0687,0.0500,0.0562,0.0625,0.0625,0.0625,0.0562,...,0.1125,0.0875,0.0687,0.0625,0.0562,0.0625,0.0625,0.0687,1,1
4,3/5/2021,0.0625,0.0625,0.0500,0.0625,0.0687,0.0625,0.0687,0.0562,0.0500,...,0.0812,0.0562,0.0687,0.0625,0.0625,0.0625,0.0562,0.0500,1,1


In [4]:
df.isna().sum()

read_date      0
interval_1     0
interval_2     0
interval_3     0
interval_4     0
interval_5     0
interval_6     0
interval_7     0
interval_8     0
interval_9     0
interval_10    0
interval_11    0
interval_12    0
interval_13    0
interval_14    0
interval_15    0
interval_16    0
interval_17    0
interval_18    0
interval_19    0
interval_20    0
interval_21    0
interval_22    0
interval_23    0
interval_24    0
interval_25    0
interval_26    0
interval_27    0
interval_28    0
interval_29    0
interval_30    0
interval_31    0
interval_32    0
interval_33    0
interval_34    0
interval_35    0
interval_36    0
interval_37    0
interval_38    0
interval_39    0
interval_40    0
interval_41    0
interval_42    0
interval_43    0
interval_44    0
interval_45    0
interval_46    0
interval_47    0
interval_48    0
id             0
label          0
dtype: int64

In [5]:
df.dtypes

read_date       object
interval_1     float64
interval_2     float64
interval_3     float64
interval_4     float64
interval_5     float64
interval_6     float64
interval_7     float64
interval_8     float64
interval_9     float64
interval_10    float64
interval_11    float64
interval_12    float64
interval_13    float64
interval_14    float64
interval_15    float64
interval_16    float64
interval_17    float64
interval_18    float64
interval_19    float64
interval_20    float64
interval_21    float64
interval_22    float64
interval_23    float64
interval_24    float64
interval_25    float64
interval_26    float64
interval_27    float64
interval_28    float64
interval_29    float64
interval_30    float64
interval_31    float64
interval_32    float64
interval_33    float64
interval_34    float64
interval_35    float64
interval_36    float64
interval_37    float64
interval_38    float64
interval_39    float64
interval_40    float64
interval_41    float64
interval_42    float64
interval_43

In [5]:
df['read_date']= pd.to_datetime(df['read_date'])
df['read_date'].head()

ValueError: unconverted data remains when parsing with format "%m/%d/%Y": " 0:00", at position 46. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [6]:
df['read_date'] = df['read_date'].str.replace(' 0:00', '')

In [7]:
df['read_date']= pd.to_datetime(df['read_date'])
df['read_date'].head()

0   2021-03-01
1   2021-03-02
2   2021-03-03
3   2021-03-04
4   2021-03-05
Name: read_date, dtype: datetime64[ns]

In [9]:
df.dtypes

read_date      datetime64[ns]
interval_1            float64
interval_2            float64
interval_3            float64
interval_4            float64
interval_5            float64
interval_6            float64
interval_7            float64
interval_8            float64
interval_9            float64
interval_10           float64
interval_11           float64
interval_12           float64
interval_13           float64
interval_14           float64
interval_15           float64
interval_16           float64
interval_17           float64
interval_18           float64
interval_19           float64
interval_20           float64
interval_21           float64
interval_22           float64
interval_23           float64
interval_24           float64
interval_25           float64
interval_26           float64
interval_27           float64
interval_28           float64
interval_29           float64
interval_30           float64
interval_31           float64
interval_32           float64
interval_3

In [10]:
df.to_csv()

,read_date,interval_1,interval_2,interval_3,interval_4,interval_5,interval_6,interval_7,interval_8,interval_9,...,interval_41,interval_42,interval_43,interval_44,interval_45,interval_46,interval_47,interval_48,id,label
0,2021-03-01,0.0625,0.0500,0.0687,0.0750,0.0687,0.0500,0.0625,0.0687,0.0687,...,0.0812,0.0687,0.0687,0.0562,0.0562,0.0687,0.0687,0.0625,1,1
1,2021-03-02,0.0625,0.0500,0.0687,0.0625,0.0625,0.0562,0.0562,0.0625,0.0687,...,0.1375,0.0750,0.0687,0.0625,0.0625,0.0562,0.0625,0.0625,1,1
2,2021-03-03,0.0625,0.0687,0.0500,0.0562,0.0687,0.0625,0.0625,0.0625,0.0500,...,0.1875,0.1062,0.0750,0.0687,0.0687,0.0625,0.0562,0.0562,1,1
3,2021-03-04,0.0625,0.0625,0.0687,0.0500,0.0562,0.0625,0.0625,0.0625,0.0562,...,0.1125,0.0875,0.0687,0.0625,0.0562,0.0625,0.0625,0.0687,1,1
4,2021-03-05,0.0625,0.0625,0.0500,0.0625,0.0687,0.0625,0.0687,0.0562,0.0500,...,0.0812,0.0562,0.0687,0.0625,0.0625,0.0625,0.0562,0.0500,1,1


In [8]:
df['day_of_week_num'] = df['read_date'].dt.day_of_week
df['day_of_month'] = df['read_date'].dt.day
df['month'] = df['read_date'].dt.month
df.head()

,read_date,interval_1,interval_2,interval_3,interval_4,interval_5,interval_6,interval_7,interval_8,interval_9,...,interval_44,interval_45,interval_46,interval_47,interval_48,id,label,day_of_week_num,day_of_month,month
0,2021-03-01,0.0625,0.0500,0.0687,0.0750,0.0687,0.0500,0.0625,0.0687,0.0687,...,0.0562,0.0562,0.0687,0.0687,0.0625,1,1,0,1,3
1,2021-03-02,0.0625,0.0500,0.0687,0.0625,0.0625,0.0562,0.0562,0.0625,0.0687,...,0.0625,0.0625,0.0562,0.0625,0.0625,1,1,1,2,3
2,2021-03-03,0.0625,0.0687,0.0500,0.0562,0.0687,0.0625,0.0625,0.0625,0.0500,...,0.0687,0.0687,0.0625,0.0562,0.0562,1,1,2,3,3
3,2021-03-04,0.0625,0.0625,0.0687,0.0500,0.0562,0.0625,0.0625,0.0625,0.0562,...,0.0625,0.0562,0.0625,0.0625,0.0687,1,1,3,4,3
4,2021-03-05,0.0625,0.0625,0.0500,0.0625,0.0687,0.0625,0.0687,0.0562,0.0500,...,0.0625,0.0625,0.0625,0.0562,0.0500,1,1,4,5,3


In [9]:
df2= pd.read_csv('../data/EV_data_clusters_added.csv')
df2.head()

,id,interval_1,interval_2,interval_3,interval_4,interval_5,interval_6,interval_7,interval_8,interval_9,...,interval_40,interval_41,interval_42,interval_43,interval_44,interval_45,interval_46,interval_47,interval_48,power-usage-class
0,1,0.357178,0.328778,0.167091,0.146717,0.146033,0.143589,0.148480,0.148617,0.146985,...,0.145767,0.308674,0.280676,0.447120,0.427693,0.420761,0.421170,0.398348,0.397659,2
1,2,0.931628,0.780407,0.701607,0.587072,0.419270,0.347533,0.226337,0.131765,0.116554,...,0.302152,0.280278,0.272800,0.271035,0.253641,0.543993,0.817770,0.901472,0.997396,0
2,3,0.624157,0.633398,0.628098,0.621033,0.582048,0.544943,0.530141,0.471989,0.403635,...,0.686526,0.807307,0.792509,0.769272,0.737748,0.796576,0.727561,0.608802,0.557446,3
3,4,0.367772,0.401193,0.398485,0.398617,0.338289,0.298198,0.239520,0.239241,0.246991,...,0.499426,0.543993,0.676065,0.690196,0.672387,0.625657,0.464509,0.381902,0.412065,0
4,5,0.139646,0.142496,0.140054,0.131641,0.134761,0.135709,0.129870,0.134491,0.133946,...,0.313426,0.319811,0.319135,0.281502,0.238013,0.204322,0.186928,0.163563,0.150385,2


In [10]:
merged_df = pd.merge(df,df2[['id','power-usage-class']], on='id',how='left')
merged_df.head()

,read_date,interval_1,interval_2,interval_3,interval_4,interval_5,interval_6,interval_7,interval_8,interval_9,...,interval_45,interval_46,interval_47,interval_48,id,label,day_of_week_num,day_of_month,month,power-usage-class
0,2021-03-01,0.0625,0.0500,0.0687,0.0750,0.0687,0.0500,0.0625,0.0687,0.0687,...,0.0562,0.0687,0.0687,0.0625,1,1,0,1,3,2
1,2021-03-02,0.0625,0.0500,0.0687,0.0625,0.0625,0.0562,0.0562,0.0625,0.0687,...,0.0625,0.0562,0.0625,0.0625,1,1,1,2,3,2
2,2021-03-03,0.0625,0.0687,0.0500,0.0562,0.0687,0.0625,0.0625,0.0625,0.0500,...,0.0687,0.0625,0.0562,0.0562,1,1,2,3,3,2
3,2021-03-04,0.0625,0.0625,0.0687,0.0500,0.0562,0.0625,0.0625,0.0625,0.0562,...,0.0562,0.0625,0.0625,0.0687,1,1,3,4,3,2
4,2021-03-05,0.0625,0.0625,0.0500,0.0625,0.0687,0.0625,0.0687,0.0562,0.0500,...,0.0625,0.0625,0.0562,0.0500,1,1,4,5,3,2


In [ ]:
df= merged_df

In [11]:
unique_ids = df['id'].unique()
unique_ids

array([ 1,  2, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 24, 23, 22, 11,  3,  4,  5,  6,  7,  8,  9, 10, 12,
       21, 13, 14, 15, 16, 17, 18, 19, 20, 44, 50, 86, 59, 78, 49, 72, 54,
       75, 53, 66, 79, 70, 60, 74, 62, 52, 45, 77, 63, 68, 47, 64, 69, 84,
       48, 82, 80, 55, 67, 81, 71, 58, 51, 65, 76, 61, 85, 56, 88, 46, 87,
       83, 73, 57])

In [12]:
from sklearn.model_selection import train_test_split
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)

In [13]:
train_data = df.loc[df['id'].isin(train_ids)]
test_data = df.loc[df['id'].isin(test_ids)]

In [14]:
X_train = train_data.drop(["id","label"],axis=1)
y_train = train_data["label"]
X_test = test_data.drop(["id","label"],axis=1)
y_test = test_data["label"]
X_train.head()

,read_date,interval_1,interval_2,interval_3,interval_4,interval_5,interval_6,interval_7,interval_8,interval_9,...,interval_42,interval_43,interval_44,interval_45,interval_46,interval_47,interval_48,day_of_week_num,day_of_month,month
46,2021-03-01,0.1562,0.1562,0.1500,0.1437,0.1187,0.1125,0.1312,0.1625,0.1875,...,0.2125,0.2125,0.1812,0.1812,1.2437,1.3500,1.7812,0,1,3
47,2021-03-02,1.3437,1.3687,1.3937,1.3625,1.3187,1.5437,0.2250,0.1187,0.1500,...,0.1437,0.1437,0.1562,0.0875,0.8875,1.6687,1.3875,1,2,3
48,2021-03-03,1.7312,1.3500,1.3062,1.3125,0.2125,0.0687,0.1000,0.0875,0.0562,...,0.1187,0.1562,0.1375,0.1562,0.9750,1.3000,1.5625,2,3,3
49,2021-03-04,1.2937,1.6875,1.3000,1.3187,1.3250,1.2875,0.5937,0.0625,0.0750,...,0.1312,0.1187,0.2312,0.0937,0.9062,1.3250,1.6437,3,4,3
50,2021-03-05,1.3375,1.8000,1.3125,1.2875,1.3062,1.3000,0.3937,0.0812,0.1000,...,0.0750,0.0625,0.0437,0.0875,0.1000,0.0750,0.1000,4,5,3


In [17]:
X_test.drop('read_date',axis=1,inplace=True)
X_train.drop('read_date',axis=1,inplace=True)
X_train.head()

,interval_1,interval_2,interval_3,interval_4,interval_5,interval_6,interval_7,interval_8,interval_9,interval_10,...,interval_42,interval_43,interval_44,interval_45,interval_46,interval_47,interval_48,day_of_week_num,day_of_month,month
46,0.1562,0.1562,0.1500,0.1437,0.1187,0.1125,0.1312,0.1625,0.1875,0.1000,...,0.2125,0.2125,0.1812,0.1812,1.2437,1.3500,1.7812,0,1,3
47,1.3437,1.3687,1.3937,1.3625,1.3187,1.5437,0.2250,0.1187,0.1500,0.1375,...,0.1437,0.1437,0.1562,0.0875,0.8875,1.6687,1.3875,1,2,3
48,1.7312,1.3500,1.3062,1.3125,0.2125,0.0687,0.1000,0.0875,0.0562,0.0500,...,0.1187,0.1562,0.1375,0.1562,0.9750,1.3000,1.5625,2,3,3
49,1.2937,1.6875,1.3000,1.3187,1.3250,1.2875,0.5937,0.0625,0.0750,0.0812,...,0.1312,0.1187,0.2312,0.0937,0.9062,1.3250,1.6437,3,4,3
50,1.3375,1.8000,1.3125,1.2875,1.3062,1.3000,0.3937,0.0812,0.1000,0.0500,...,0.0750,0.0625,0.0437,0.0875,0.1000,0.0750,0.1000,4,5,3


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [19]:
# Set up the KFold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store the scores from each fold
train_scores_lr = []
val_scores_lr = []
train_scores_rf = []
val_scores_rf = []
train_scores_knn = []
val_scores_knn = []
train_scores_xgb = []
val_scores_xgb = []

# Iterate over the folds
for train_idx, val_idx in kfold.split(X_train, y_train):
    # Split the training data into train and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # Create the models
    lr_model = LogisticRegression(random_state=42)
    rf_model = RandomForestClassifier(random_state=42)
    knn_model = KNeighborsClassifier()
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # Train the models on the training fold
    lr_model.fit(X_train_fold, y_train_fold)
    rf_model.fit(X_train_fold, y_train_fold)
    knn_model.fit(X_train_fold, y_train_fold)
    xgb_model.fit(X_train_fold, y_train_fold)

    # Evaluate the models on the validation fold
    train_score_lr = accuracy_score(y_train_fold, lr_model.predict(X_train_fold))
    val_score_lr = accuracy_score(y_val_fold, lr_model.predict(X_val_fold))

    train_score_rf = accuracy_score(y_train_fold, rf_model.predict(X_train_fold))
    val_score_rf = accuracy_score(y_val_fold, rf_model.predict(X_val_fold))

    train_score_knn = accuracy_score(y_train_fold, knn_model.predict(X_train_fold))
    val_score_knn = accuracy_score(y_val_fold, knn_model.predict(X_val_fold))

    train_score_xgb = accuracy_score(y_train_fold, xgb_model.predict(X_train_fold))
    val_score_xgb = accuracy_score(y_val_fold, xgb_model.predict(X_val_fold))

    # Append the scores to the lists
    train_scores_lr.append(train_score_lr)
    val_scores_lr.append(val_score_lr)
    train_scores_rf.append(train_score_rf)
    val_scores_rf.append(val_score_rf)
    train_scores_knn.append(train_score_knn)
    val_scores_knn.append(val_score_knn)
    train_scores_xgb.append(train_score_xgb)
    val_scores_xgb.append(val_score_xgb)

    print(f'Fold {len(train_scores_lr)}:')
    print(f'Logistic Regression: Train Accuracy = {train_score_lr:.4f}, Validation Accuracy = {val_score_lr:.4f}')
    print(f'Random Forest: Train Accuracy = {train_score_rf:.4f}, Validation Accuracy = {val_score_rf:.4f}')
    print(f'KNN: Train Accuracy = {train_score_knn:.4f}, Validation Accuracy = {val_score_knn:.4f}')
    print(f'XGBoost: Train Accuracy = {train_score_xgb:.4f}, Validation Accuracy = {val_score_xgb:.4f}')
    # Print the mean scores
print('\nMean Scores:')
print(f'Logistic Regression: Mean Train Accuracy = {sum(train_scores_lr) / len(train_scores_lr):.4f}, Mean Validation Accuracy = {sum(val_scores_lr) / len(val_scores_lr):.4f}')
print(f'Random Forest: Mean Train Accuracy = {sum(train_scores_rf) / len(train_scores_rf):.4f}, Mean Validation Accuracy = {sum(val_scores_rf) / len(val_scores_rf):.4f}')
print(f'KNN: Mean Train Accuracy = {sum(train_scores_knn) / len(train_scores_knn):.4f}, Mean Validation Accuracy = {sum(val_scores_knn) / len(val_scores_knn):.4f}')
print(f'XGBoost: Mean Train Accuracy = {sum(train_scores_xgb) / len(train_scores_xgb):.4f}, Mean Validation Accuracy = {sum(val_scores_xgb) / len(val_scores_xgb):.4f}')

# Train the final models on the entire training set
final_lr_model = LogisticRegression(random_state=42)
final_rf_model = RandomForestClassifier(random_state=42)
final_knn_model = KNeighborsClassifier()
final_xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

final_lr_model.fit(X_train, y_train)
final_rf_model.fit(X_train, y_train)
final_knn_model.fit(X_train, y_train)
final_xgb_model.fit(X_train, y_train)

# Evaluate the final models on the test set
test_accuracy_lr = accuracy_score(y_test, final_lr_model.predict(X_test))
test_accuracy_rf = accuracy_score(y_test, final_rf_model.predict(X_test))
test_accuracy_knn = accuracy_score(y_test, final_knn_model.predict(X_test))
test_accuracy_xgb = accuracy_score(y_test, final_xgb_model.predict(X_test))

print('\nTest Accuracies:')
print(f'Logistic Regression: {test_accuracy_lr:.4f}')
print(f'Random Forest: {test_accuracy_rf:.4f}')
print(f'KNN: {test_accuracy_knn:.4f}')
print(f'XGBoost: {test_accuracy_xgb:.4f}')

/Users/bijubiju/Desktop/ml projects/current/Transformer/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 1:
Logistic Regression: Train Accuracy = 0.7100, Validation Accuracy = 0.7298
Random Forest: Train Accuracy = 1.0000, Validation Accuracy = 0.9255
KNN: Train Accuracy = 0.8436, Validation Accuracy = 0.7640
XGBoost: Train Accuracy = 1.0000, Validation Accuracy = 0.9441


/Users/bijubiju/Desktop/ml projects/current/Transformer/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 2:
Logistic Regression: Train Accuracy = 0.7205, Validation Accuracy = 0.6894
Random Forest: Train Accuracy = 1.0000, Validation Accuracy = 0.8913
KNN: Train Accuracy = 0.8502, Validation Accuracy = 0.7640
XGBoost: Train Accuracy = 1.0000, Validation Accuracy = 0.9208


/Users/bijubiju/Desktop/ml projects/current/Transformer/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

Fold 4:
Logistic Regression: Train Accuracy = 0.7275, Validation Accuracy = 0.6957
Random Forest: Train Accuracy = 1.0000, Validation Accuracy = 0.9255
KNN: Train Accuracy = 0.8358, Validation Accuracy = 0.7748
XGBoost: Train Accuracy = 1.0000, Validation Accuracy = 0.9457


/Users/bijubiju/Desktop/ml projects/current/Transformer/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 5:
Logistic Regression: Train Accuracy = 0.7147, Validation Accuracy = 0.6863
Random Forest: Train Accuracy = 1.0000, Validation Accuracy = 0.9053
KNN: Train Accuracy = 0.8525, Validation Accuracy = 0.7562
XGBoost: Train Accuracy = 1.0000, Validation Accuracy = 0.9146

Mean Scores:
Logistic Regression: Mean Train Accuracy = 0.7183, Mean Validation Accuracy = 0.7012
Random Forest: Mean Train Accuracy = 1.0000, Mean Validation Accuracy = 0.9102
KNN: Mean Train Accuracy = 0.8464, Mean Validation Accuracy = 0.7674
XGBoost: Mean Train Accuracy = 1.0000, Mean Validation Accuracy = 0.9301


/Users/bijubiju/Desktop/ml projects/current/Transformer/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Test Accuracies:
Logistic Regression: 0.6449
Random Forest: 0.8273
KNN: 0.8068
XGBoost: 0.8309
